In [5]:
import numpy as np
import cv2
import os

In [6]:
def feature_object_detection(template_img, template_gray, query_img, query_gray, min_match_number) :
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.7*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 10.0) # H RANSAC
        # get the bounding box around template image
        h, w = template_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv2.perspectiveTransform(template_box, H)

        detected_img = cv2.polylines(query_img, [np.int32(transformed_box)], True, (255,0,0), 3, cv2.LINE_AA)
        

        return detected_img
    else :
        print('Keypoints not enough')
        return query_img

In [7]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [8]:
vid = cv2.VideoCapture('./left_output-1.avi')

while vid.isOpened() :
    ret, frame = vid.read()
    template_img = cv2.imread('./Template-1.png')
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2BGRA)
    detected =  feature_object_detection(template_img, template_gray, frame_gray, frame, 10)
    
    if ret :
        cv2.imshow('Video frame', detected)

        if cv2.waitKey(int(1000/24)) & 0xFF == ord('q') : # this line control the period between image frame
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()